In [18]:
from typing import Any, TypedDict
from langchain.chat_models import init_chat_model
from langchain_core.messages import AnyMessage
from langchain_core.messages.utils import count_tokens_approximately
from langgraph.graph import StateGraph, START, MessageGraph, MessagesState
from langgraph.checkpoint.memory import InMemorySaver
from langmem.short_term import SummarizationNode

In [19]:
model = init_chat_model(model='gpt-4o')

In [ ]:
summarization_model = model.bind(max_tokens=128)

In [21]:
class State(MessagesState):
    context: dict[str, Any]

In [22]:
class LLMInputState(TypedDict):
    summarized_messages: list[AnyMessage]
    context: dict[str, Any]

In [30]:
summarization_node = SummarizationNode(
    model=summarization_model, 
    max_tokens=256,
    max_tokens_before_summary=123,
    max_summary_tokens=128,
    token_counter=count_tokens_approximately
)

In [31]:
def call_model(state: LLMInputState):
    response = model.invoke(state["summarized_messages"])
    return {"messages": response}

In [32]:
checkpointer = InMemorySaver()
builder = StateGraph(State)
builder.add_node("summarize", summarization_node)
builder.add_node("call_model", call_model)
builder.add_edge(START, "summarize")
builder.add_edge("summarize", "call_model")
graph = builder.compile(checkpointer=checkpointer)

In [33]:
config = {"configurable": {"thread_id": 1}}

In [34]:
final_response = graph.invoke({"messages": "hi, my name is bob"}, config=config)

In [35]:
final_response = graph.invoke({"messages": "write a short poem about cats"}, config=config)

In [36]:
final_response = graph.invoke({"messages": "What is my name"}, config=config)

TypeError: Completions.create() got an unexpected keyword argument 'max_token'